In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'challenge_user', # имя пользователя
             'pwd': 'aT00GAh944YU4Q1J4zbzryW6Yrz11iq54coz', # пароль
             'host': 'rc1a-eqy3nymy2wp6tbx2.mdb.yandexcloud.net', # адрес сервера
             'port': 6432, # порт подключения
             'db': 'data-analyst-challenge'}# название базы данных
 
# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                             db_config['pwd'],
                                             db_config['host'],
                                             db_config['port'],
                                             db_config['db'])
# Подключаемся к БД.
engine = create_engine(connection_string)

# Формируем sql-запрос(пример).
query = ''' WITH cte AS (
  SELECT 
    finished_lesson_test.lesson_id, 
    user_id, 
    profession_id, 
    profession_name, 
    lesson_name, 
    date_created, 
    LAG(date_created) OVER (PARTITION BY user_id, profession_id ORDER BY date_created) AS prev_lesson_date
  FROM 
    finished_lesson_test 
    JOIN lesson_index_test ON finished_lesson_test.lesson_id = lesson_index_test.lesson_id 
  WHERE 
    lesson_index_test.profession_name = 'data-analyst' 
    AND EXTRACT(year FROM date_created) = 2020 
    AND EXTRACT(month FROM date_created) = 4
)
SELECT 
  EXTRACT(SECOND FROM (date_created - prev_lesson_date)) as delta_seconds,
prev_lesson_date,
  date_created, 
  lesson_id, 
  profession_name, 
  user_id
FROM 
  cte 
WHERE 
  date_created - prev_lesson_date <= INTERVAL '5 seconds';

        '''

# Выполняем запрос и сохраняем результат
# выполнения в DataFrame.
# Sqlalchemy автоматически установит названия колонок
# такими же, как у таблицы в БД.
data_raw = pd.io.sql.read_sql(query, con = engine)


In [9]:
data_raw.head()

,delta_seconds,prev_lesson_date,date_created,lesson_id,profession_name,user_id
0,4.357835,2020-04-16 14:41:54.781518+00:00,2020-04-16 14:41:59.139353+00:00,3891bd65-5ff9-4355-b886-f5796ce6ba72,data-analyst,4409
1,4.457578,2020-04-12 13:27:55.500944+00:00,2020-04-12 13:27:59.958522+00:00,cec6f205-490c-4e36-8167-53db63585876,data-analyst,16144
2,4.515751,2020-04-12 13:27:59.958522+00:00,2020-04-12 13:28:04.474273+00:00,b1505742-72ef-456f-b3f6-932461fadce6,data-analyst,16144
3,4.073750,2020-04-12 13:28:04.474273+00:00,2020-04-12 13:28:08.548023+00:00,2a608a6c-97a2-45e2-996a-a215c15756e8,data-analyst,16144
4,3.927094,2020-04-12 13:36:33.836217+00:00,2020-04-12 13:36:37.763311+00:00,d8447ed8-0b62-4107-8b6a-10ce6fd8fbd7,data-analyst,16144


In [20]:
# Правда непонятно почему в задании указано вывести lesson_id


import plotly.express as px
# проверяем, есть ли уроки с слишком быстрым временем прохождения
if data_raw.empty:
    print('Нет уроков с слишком быстрым временем прохождения')
else:
    # считаем количество таких уроков
    num_fast_lessons = len(data_raw)
    # выводим наименования уроков
    fast_lesson_ids = list(data_raw['lesson_id'].unique())
    print('Есть уроки с слишком быстрым временем прохождения:')
    print(fast_lesson_ids)
    # создаем интерактивный график для визуализации распределения времени прохождения уроков
    fig = px.histogram(data_raw, x='delta_seconds', nbins=30,
                       title='Распределение времени прохождения уроков')
    fig.show()

Есть уроки с слишком быстрым временем прохождения:
['3891bd65-5ff9-4355-b886-f5796ce6ba72', 'cec6f205-490c-4e36-8167-53db63585876', 'b1505742-72ef-456f-b3f6-932461fadce6', '2a608a6c-97a2-45e2-996a-a215c15756e8', 'd8447ed8-0b62-4107-8b6a-10ce6fd8fbd7', '47cf9db0-4be1-412e-80f7-a3971b6b3c88', 'd0162814-ae0a-4c0c-8e60-22ecf53c5348', 'c43465c3-1579-43d2-b229-0c0eaec20e27', '21fbfb7c-8b74-40e7-9cd8-9590b7903f26', '2fb8f1df-6e9a-4b1f-86ff-ca213ef98cc7', 'dbb9697d-3d32-4b81-ad1c-c9ac0496ee39', '6a4be97b-048a-43d8-b8f0-964f52ac7d30', '24d355f6-4f88-4c05-bf51-e0e1db42f184', 'cd601912-2c45-429c-b9f1-3ef16fb07a31', 'a7b2c524-eef0-46af-b065-044492a4f4c0', 'e1324af1-8285-4c22-93f0-10e7641dc254', 'cc57bb5d-707b-4466-8c6b-651d9447dfe5', '83b099db-0830-4bc6-b6ec-fe25a29a4e42', '35bef149-e188-4385-8622-2c73911dfc86', '86fbb98c-0003-4cdd-80cd-55fadc110334', 'efcdf907-60ff-41d2-aacb-7789d9c2f443', 'ba6241c8-28da-4035-863f-400500d397e7', '19c16deb-5db3-43d7-96c3-49ee749c0649', '55db68d2-3657-422c-aa60-099